In [2]:
import torch
import torchvision
from torchvision import transforms

# Define the transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load the data
train_data = torchvision.datasets.ImageFolder(root="train/", transform=transform)
test_data = torchvision.datasets.ImageFolder(root="test/", transform=transform)

# Define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

In [3]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

In [4]:
# Define the model
model = resnet50(pretrained=True)

# Replace the last layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_data.classes))

d:\AI\Tortilla Price Prediction\tortilaenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\AI\Tortilla Price Prediction\tortilaenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ephre/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [5]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Move the model to the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [6]:
# Define the number of epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):
    # Train the model on the training set
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Move the data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update the training loss
        train_loss += loss.item() * inputs.size(0)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            # Move the data to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Update the test loss and accuracy
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            test_acc += torch.sum(preds == labels.data)

    # Print the training and test loss and accuracy
    train_loss /= len(train_data)
    test_loss /= len(test_data)
    test_acc = test_acc.double() / len(test_data)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")

Epoch [1/10] Train Loss: 1.0535 Test Loss: 0.7455 Test Acc: 0.6974
Epoch [2/10] Train Loss: 0.5238 Test Loss: 0.4993 Test Acc: 0.8026
Epoch [3/10] Train Loss: 0.2903 Test Loss: 0.4683 Test Acc: 0.8275
Epoch [4/10] Train Loss: 0.1386 Test Loss: 0.4179 Test Acc: 0.8330
Epoch [5/10] Train Loss: 0.0809 Test Loss: 0.4424 Test Acc: 0.8321
Epoch [6/10] Train Loss: 0.0494 Test Loss: 0.4549 Test Acc: 0.8413
Epoch [7/10] Train Loss: 0.0258 Test Loss: 0.4674 Test Acc: 0.8487
Epoch [8/10] Train Loss: 0.0286 Test Loss: 0.4592 Test Acc: 0.8487
Epoch [9/10] Train Loss: 0.0331 Test Loss: 0.4695 Test Acc: 0.8459
Epoch [10/10] Train Loss: 0.0156 Test Loss: 0.4877 Test Acc: 0.8450


In [8]:
torch.save(model, 'resnet50_dog_emotion.pth')

In [9]:
model = torch.load('resnet50_dog_emotion.pth')

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Ensure the same transformations as during training
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming 'model' is already defined and fine-tuned as per your script
model.eval()
model = model.to(device)

# Load and preprocess the image
image_path = 'path_to_your_image.jpg'  # Replace with the actual path to your image
image = Image.open(image_path)
image = transform(image)
image = image.unsqueeze(0)  # Add batch dimension

# Move the image to the device
image = image.to(device)

# Perform inference
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

# Convert the predicted index to the corresponding class label
predicted_class = train_data.classes[predicted.item()]
print(f'Predicted class: {predicted_class}')
